# Load libraries

In [1]:
library(data.table)

# Load single-cell data (Park et al)

In [2]:
# load reference (Park) data
park_data_path <- '/mnt/disk1/nbarkas/deconvolution_method/datasets/Park_etal/GSE107585_Mouse_kidney_single_cell_datamatrix.txt'
park_data <- fread(park_data_path)
park_data <- as.data.frame(park_data)

Warning message in fread(park_data_path):
“Detected 43745 column names but the data has 43746 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”


In [3]:
# inspect loaded data
park_data[1:3,1:3]

,V1,AAACCTGAGATATGCA-1,AAACCTGGTTGTGGCC-1
,<chr>,<int>,<int>
1,Cluster_Number,3,5
2,Rp1,0,0
3,Sox17,0,0


In [4]:
# Get a cell cluster factor
park_cluster <- as.character(t(park_data[1,])[,1])
names(park_cluster) <- colnames(park_data)
park_cluster <- park_cluster[-1]
park_cluster <- as.factor(park_cluster)

In [5]:
head(park_cluster)

AAACCTGAGATATGCA-1 AAACCTGGTTGTGGCC-1 AAACGGGAGCGTCTAT-1 AAACGGGAGCTCCTTC-1 
                 3                  5                  3                  3 
AAACGGGCACCTCGGA-1 AAACGGGCAGGTGCCT-1 
                 3                  5 
Levels: 1 10 11 12 13 14 15 16 2 3 4 5 6 7 8 9

In [6]:
# remove the cluster number row
park_tmp <- park_data[-1,]

In [7]:
park_tmp[1:3,1:3]

,V1,AAACCTGAGATATGCA-1,AAACCTGGTTGTGGCC-1
,<chr>,<int>,<int>
2,Rp1,0,0
3,Sox17,0,0
4,Mrpl15,0,0


In [8]:
park_tmp <- as.data.frame(park_tmp)

In [9]:
rownames(park_tmp) <- park_tmp$V1

In [10]:
park_tmp[1:3,1:3]

,V1,AAACCTGAGATATGCA-1,AAACCTGGTTGTGGCC-1
,<chr>,<int>,<int>
Rp1,Rp1,0,0
Sox17,Sox17,0,0
Mrpl15,Mrpl15,0,0


In [11]:
# Remove V1 columnsb
park_tmp <- park_tmp[,-1]
park_tmp[1:3,1:3]

,AAACCTGAGATATGCA-1,AAACCTGGTTGTGGCC-1,AAACGGGAGCGTCTAT-1
,<int>,<int>,<int>
Rp1,0,0,0
Sox17,0,0,0
Mrpl15,0,0,0


In [12]:
park_counts.m <- as.matrix(park_tmp)
park_counts.m[1:3,1:3]

,AAACCTGAGATATGCA-1,AAACCTGGTTGTGGCC-1,AAACGGGAGCGTCTAT-1
Rp1,0,0,0
Sox17,0,0,0
Mrpl15,0,0,0


# Load the bulk data to be deconvolved (Cracium)

In [13]:
# load data to be deconvolved (craciun)
path.to.data <- '/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/'
files <- list.files(path.to.data)

In [14]:
files

[1] "GSM1591197_Normalrep1.rawcounts.txt" 
 [2] "GSM1591198_Normalrep2.rawcounts.txt" 
 [3] "GSM1591199_Normalrep3.rawcounts.txt" 
 [4] "GSM1591200_FA1dayrep1.rawcounts.txt" 
 [5] "GSM1591201_FA1dayrep2.rawcounts.txt" 
 [6] "GSM1591202_FA1dayrep3.rawcounts.txt" 
 [7] "GSM1591203_FA2dayrep1.rawcounts.txt" 
 [8] "GSM1591204_FA2dayrep2.rawcounts.txt" 
 [9] "GSM1591205_FA2dayrep3.rawcounts.txt" 
[10] "GSM1591206_FA3dayrep1.rawcounts.txt" 
[11] "GSM1591207_FA3dayrep2.rawcounts.txt" 
[12] "GSM1591208_FA3dayrep3.rawcounts.txt" 
[13] "GSM1591209_FA7dayrep1.rawcounts.txt" 
[14] "GSM1591210_FA7dayrep2.rawcounts.txt" 
[15] "GSM1591211_FA7dayrep3.rawcounts.txt" 
[16] "GSM1591212_FA14dayrep1.rawcounts.txt"
[17] "GSM1591213_FA14dayrep2.rawcounts.txt"
[18] "GSM1591214_FA14dayrep3.rawcounts.txt"

In [15]:
names(files) <- nbHelpers::strpart(files,'.',1,fixed=TRUE)
head(files)

Warning message:
“replacing previous import ‘Rcpp::prompt’ by ‘utils::prompt’ when loading ‘nbHelpers’”
Warning message:
“replacing previous import ‘Rcpp::.DollarNames’ by ‘utils::.DollarNames’ when loading ‘nbHelpers’”
Warning message:
“replacing previous import ‘Matrix::tail’ by ‘utils::tail’ when loading ‘nbHelpers’”
Warning message:
“replacing previous import ‘Matrix::head’ by ‘utils::head’ when loading ‘nbHelpers’”


GSM1591197_Normalrep1                 GSM1591198_Normalrep2 
"GSM1591197_Normalrep1.rawcounts.txt" "GSM1591198_Normalrep2.rawcounts.txt" 
                GSM1591199_Normalrep3                 GSM1591200_FA1dayrep1 
"GSM1591199_Normalrep3.rawcounts.txt" "GSM1591200_FA1dayrep1.rawcounts.txt" 
                GSM1591201_FA1dayrep2                 GSM1591202_FA1dayrep3 
"GSM1591201_FA1dayrep2.rawcounts.txt" "GSM1591202_FA1dayrep3.rawcounts.txt"

In [16]:
full.files <- paste0(path.to.data, files)
head(full.files)

[1] "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591197_Normalrep1.rawcounts.txt"
[2] "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591198_Normalrep2.rawcounts.txt"
[3] "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591199_Normalrep3.rawcounts.txt"
[4] "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591200_FA1dayrep1.rawcounts.txt"
[5] "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591201_FA1dayrep2.rawcounts.txt"
[6] "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591202_FA1dayrep3.rawcounts.txt"

In [17]:
names(full.files) <- names(files)
full.files

GSM1591197_Normalrep1 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591197_Normalrep1.rawcounts.txt" 
                                                                                   GSM1591198_Normalrep2 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591198_Normalrep2.rawcounts.txt" 
                                                                                   GSM1591199_Normalrep3 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591199_Normalrep3.rawcounts.txt" 
                                                                                   GSM1591200_FA1dayrep1 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591200_FA1dayrep1.rawcounts.txt" 
                                                                                   GSM1591201_FA1dayrep2 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591201_FA1dayrep2.rawcounts.txt" 
                                                                                   GSM1591202_FA1dayrep3 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591202_FA1dayrep3.rawcounts.txt" 
                                                                                   GSM1591203_FA2dayrep1 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591203_FA2dayrep1.rawcounts.txt" 
                                                                                   GSM1591204_FA2dayrep2 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591204_FA2dayrep2.rawcounts.txt" 
                                                                                   GSM1591205_FA2dayrep3 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591205_FA2dayrep3.rawcounts.txt" 
                                                                                   GSM1591206_FA3dayrep1 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591206_FA3dayrep1.rawcounts.txt" 
                                                                                   GSM1591207_FA3dayrep2 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591207_FA3dayrep2.rawcounts.txt" 
                                                                                   GSM1591208_FA3dayrep3 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591208_FA3dayrep3.rawcounts.txt" 
                                                                                   GSM1591209_FA7dayrep1 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591209_FA7dayrep1.rawcounts.txt" 
                                                                                   GSM1591210_FA7dayrep2 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591210_FA7dayrep2.rawcounts.txt" 
                                                                                   GSM1591211_FA7dayrep3 
 "/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591211_FA7dayrep3.rawcounts.txt" 
                                                                                  GSM1591212_FA14dayrep1 
"/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591212_FA14dayrep1.rawcounts.txt" 
                                                                                  GSM1591213_FA14dayrep2 
"/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591213_FA14dayrep2.rawcounts.txt" 
                                                                                  GSM1591214_FA14dayrep3 
"/mnt/disk1/nbarkas/deconvolution_method/datasets/craciun_etal/raw/GSM1591214_FA14dayrep3.rawcounts.txt"

In [18]:
raw_files_read <- lapply(full.files, function(f) {read.table(f, header=TRUE)})

In [19]:
# Inspect loaded data
str(raw_files_read,1)

List of 18
 $ GSM1591197_Normalrep1 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591198_Normalrep2 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591199_Normalrep3 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591200_FA1dayrep1 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591201_FA1dayrep2 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591202_FA1dayrep3 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591203_FA2dayrep1 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591204_FA2dayrep2 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591205_FA2dayrep3 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591206_FA3dayrep1 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591207_FA3dayrep2 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591208_FA3dayrep3 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591209_FA7dayrep1 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591210_FA7dayrep2 :'data.frame':	37996 obs. of  2 variables:
 $ GSM1591211_FA7dayrep3 :'data.frame

In [20]:
head(raw_files_read[[1]])

,ensemblid,counts
,<chr>,<int>
1,ENSMUSG00000000001,456
2,ENSMUSG00000000003,0
3,ENSMUSG00000000028,8
4,ENSMUSG00000000031,0
5,ENSMUSG00000000037,26
6,ENSMUSG00000000049,27


In [21]:
# check that all genes are in the same order
gene_names <- lapply(raw_files_read, function(x){x$ensemblid})

In [22]:
all(unlist(lapply(seq(1,length(gene_names)), function(i) { all(gene_names[[1]] == gene_names[[i]]) })))

[1] TRUE

In [23]:
# make data.frame
craciun_tmp <- do.call(cbind,lapply(raw_files_read, function(x) {x[2]}))
rownames(craciun_tmp) <- raw_files_read[[1]]$ensemblid
colnames(craciun_tmp) <- names(raw_files_read)
craciun_counts <- as.matrix(craciun_tmp)

In [24]:
craciun_counts[1:4,1:4]

,GSM1591197_Normalrep1,GSM1591198_Normalrep2,GSM1591199_Normalrep3,GSM1591200_FA1dayrep1
ENSMUSG00000000001,456,1797,451,2291
ENSMUSG00000000003,0,0,0,0
ENSMUSG00000000028,8,7,0,38
ENSMUSG00000000031,0,0,0,10


In [25]:
n0 <- colnames(craciun_counts)
n1 <- nbHelpers::strpart(n0, '_',2)
replicate <- nbHelpers::strpart(n1, 'rep',2)

In [26]:
n0

[1] "GSM1591197_Normalrep1"  "GSM1591198_Normalrep2"  "GSM1591199_Normalrep3" 
 [4] "GSM1591200_FA1dayrep1"  "GSM1591201_FA1dayrep2"  "GSM1591202_FA1dayrep3" 
 [7] "GSM1591203_FA2dayrep1"  "GSM1591204_FA2dayrep2"  "GSM1591205_FA2dayrep3" 
[10] "GSM1591206_FA3dayrep1"  "GSM1591207_FA3dayrep2"  "GSM1591208_FA3dayrep3" 
[13] "GSM1591209_FA7dayrep1"  "GSM1591210_FA7dayrep2"  "GSM1591211_FA7dayrep3" 
[16] "GSM1591212_FA14dayrep1" "GSM1591213_FA14dayrep2" "GSM1591214_FA14dayrep3"

In [27]:
n1

[1] "Normalrep1"  "Normalrep2"  "Normalrep3"  "FA1dayrep1"  "FA1dayrep2" 
 [6] "FA1dayrep3"  "FA2dayrep1"  "FA2dayrep2"  "FA2dayrep3"  "FA3dayrep1" 
[11] "FA3dayrep2"  "FA3dayrep3"  "FA7dayrep1"  "FA7dayrep2"  "FA7dayrep3" 
[16] "FA14dayrep1" "FA14dayrep2" "FA14dayrep3"

In [28]:
replicate

[1] "1" "2" "3" "1" "2" "3" "1" "2" "3" "1" "2" "3" "1" "2" "3" "1" "2" "3"

In [29]:
timemap <- c("Normal" =0,  "FA1day"=1,  "FA2day"=2,  "FA3day" =3, "FA7day"=7,  "FA14day"=14)
craniun_sample_time <- timemap[nbHelpers::strpart(n1, 'rep',1)]
names( craniun_sample_time ) <- colnames(craciun_counts)

In [30]:
craniun_sample_time

GSM1591197_Normalrep1  GSM1591198_Normalrep2  GSM1591199_Normalrep3 
                     0                      0                      0 
 GSM1591200_FA1dayrep1  GSM1591201_FA1dayrep2  GSM1591202_FA1dayrep3 
                     1                      1                      1 
 GSM1591203_FA2dayrep1  GSM1591204_FA2dayrep2  GSM1591205_FA2dayrep3 
                     2                      2                      2 
 GSM1591206_FA3dayrep1  GSM1591207_FA3dayrep2  GSM1591208_FA3dayrep3 
                     3                      3                      3 
 GSM1591209_FA7dayrep1  GSM1591210_FA7dayrep2  GSM1591211_FA7dayrep3 
                     7                      7                      7 
GSM1591212_FA14dayrep1 GSM1591213_FA14dayrep2 GSM1591214_FA14dayrep3 
                    14                     14                     14

# Convert Identifiers

In [31]:
# Load libraries
suppressPackageStartupMessages(library(Biobase))
suppressPackageStartupMessages(library(org.Mm.eg.db))

In [32]:
columns(org.Mm.eg.db)

[1] "ACCNUM"       "ALIAS"        "ENSEMBL"      "ENSEMBLPROT"  "ENSEMBLTRANS"
 [6] "ENTREZID"     "ENZYME"       "EVIDENCE"     "EVIDENCEALL"  "GENENAME"    
[11] "GO"           "GOALL"        "IPI"          "MGI"          "ONTOLOGY"    
[16] "ONTOLOGYALL"  "PATH"         "PFAM"         "PMID"         "PROSITE"     
[21] "REFSEQ"       "SYMBOL"       "UNIGENE"      "UNIPROT"

In [33]:
rownames(park_counts.m)[1:3]

[1] "Rp1"    "Sox17"  "Mrpl15"

In [34]:
rownames(craciun_counts)[1:3]

[1] "ENSMUSG00000000001" "ENSMUSG00000000003" "ENSMUSG00000000028"

In [35]:
symbol.ids <- mapIds(x = org.Mm.eg.db, 
                  keys = rownames(craciun_counts), 
                  keytype = 'ENSEMBL', 
                  column = 'SYMBOL', multiVals = 'first')

'select()' returned 1:many mapping between keys and columns



In [36]:
head(symbol.ids)

ENSMUSG00000000001 ENSMUSG00000000003 ENSMUSG00000000028 ENSMUSG00000000031 
           "Gnai3"             "Pbsn"            "Cdc45"              "H19" 
ENSMUSG00000000037 ENSMUSG00000000049 
           "Scml2"             "Apoh"

In [37]:
rownames(craciun_counts) <- unname(symbol.ids[rownames(craciun_counts)])

In [38]:
table(duplicated(rownames(craciun_counts)))


FALSE  TRUE 
25542 12454 

In [39]:
head(rownames(craciun_counts))

[1] "Gnai3" "Pbsn"  "Cdc45" "H19"   "Scml2" "Apoh"

In [40]:
craciun_counts[1:3,1:3]

,GSM1591197_Normalrep1,GSM1591198_Normalrep2,GSM1591199_Normalrep3
Gnai3,456,1797,451
Pbsn,0,0,0
Cdc45,8,7,0


In [41]:
suppressPackageStartupMessages(library(tidyverse))

In [42]:
# summarize reads from same gene
data.table(craciun_counts,keep.rownames = TRUE) %>% group_by(rn) %>% summarize_all(funs(sum)) -> summarized_data

Warning message:
“`funs()` is deprecated as of dplyr 0.8.0.
Please use a list of either functions or lambdas: 

  # Simple named list: 
  list(mean = mean, median = median)

  # Auto named with `tibble::lst()`: 
  tibble::lst(mean, median)

  # Using lambdas
  list(~ mean(., trim = .2), ~ median(., na.rm = TRUE))
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”


In [43]:
cracium_counts_sum <- as.matrix(summarized_data[,-1])

In [44]:
rownames(cracium_counts_sum) <- summarized_data$rn

In [45]:
table(duplicated(rownames(cracium_counts_sum)))


FALSE 
25542 

In [46]:
table(rownames(cracium_counts_sum) %in% rownames(park_counts.m))


FALSE  TRUE 
11452 14090 

In [47]:
table(rownames(park_counts.m) %in% rownames(cracium_counts_sum))


FALSE  TRUE 
 2182 14090 

In [48]:
sum(cracium_counts_sum[!(rownames(cracium_counts_sum) %in% rownames(park_counts.m)),])/sum(cracium_counts_sum)

[1] 0.8363287

In [49]:
sum(park_counts.m[!(rownames(park_counts.m) %in% rownames(cracium_counts_sum)),])/sum(park_counts.m)

[1] 0.2451579

In [50]:
table(duplicated(rownames(cracium_counts_sum)))


FALSE 
25542 

In [51]:
table(duplicated(rownames(park_counts.m)))


FALSE 
16272 

In [52]:
table(is.na(rownames(cracium_counts_sum)))


FALSE  TRUE 
25541     1 

In [53]:
table(is.na(rownames(park_counts.m)))


FALSE 
16272 

In [54]:
cracium_counts_sum <- cracium_counts_sum[!is.na(rownames(cracium_counts_sum)),]

## Convert Park to expression set

In [55]:
park.metadata.df <- data.frame(sampleID=names(park_cluster),cluster=park_cluster)

In [56]:
dim(park.metadata.df)

[1] 43745     2

In [57]:
dim(park_counts.m)

[1] 16272 43745

In [58]:
head(park.metadata.df)

,sampleID,cluster
,<chr>,<fct>
AAACCTGAGATATGCA-1,AAACCTGAGATATGCA-1,3
AAACCTGGTTGTGGCC-1,AAACCTGGTTGTGGCC-1,5
AAACGGGAGCGTCTAT-1,AAACGGGAGCGTCTAT-1,3
AAACGGGAGCTCCTTC-1,AAACGGGAGCTCCTTC-1,3
AAACGGGCACCTCGGA-1,AAACGGGCACCTCGGA-1,3
AAACGGGCAGGTGCCT-1,AAACGGGCAGGTGCCT-1,5


In [59]:
park.es <- ExpressionSet(assayData = park_counts.m,
                         phenoData = AnnotatedDataFrame(park.metadata.df))

In [60]:
park.es

ExpressionSet (storageMode: lockedEnvironment)
assayData: 16272 features, 43745 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: AAACCTGAGATATGCA-1 AAACCTGGTTGTGGCC-1 ...
    TTTGTCATCTGCTGTC-7 (43745 total)
  varLabels: sampleID cluster
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
Annotation:  

In [61]:
# Filter and subsample park data
park.filter.es <- park.es[,colSums(exprs(park.es)) > 1000]
park.filter.sampled.es <- park.filter.es[,sample(ncol(exprs(park.filter.es)),10000)]

In [62]:
park.filter.sampled.es

ExpressionSet (storageMode: lockedEnvironment)
assayData: 16272 features, 10000 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: TGAGCATGTTCCGTCT-7 GACGGCTGTCTCTTTA-5 ...
    GACTGCGAGCTCAACT-5 (10000 total)
  varLabels: sampleID cluster
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
Annotation:  

## Craciun et al into a eset

In [63]:
cracium_counts_sum <- cracium_counts_sum[!is.na(rownames(cracium_counts_sum)),]

In [64]:
craciun.es <- ExpressionSet(assayData = cracium_counts_sum,
                            phenoData = AnnotatedDataFrame(data.frame(time=craniun_sample_time,
                            sampleID=names(craniun_sample_time))))

In [65]:
craciun.es

ExpressionSet (storageMode: lockedEnvironment)
assayData: 25541 features, 18 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: GSM1591197_Normalrep1 GSM1591198_Normalrep2 ...
    GSM1591214_FA14dayrep3 (18 total)
  varLabels: time sampleID
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
Annotation:  

## Run MuSiC

In [66]:
library(MuSiC)
library(xbioc)

Loading required package: nnls



In [67]:
select.ct <- c("3", "5", "6", "10", "13", "14", "4", "1", "7", "12", "11", "15", "9", "8", "16", "2")

In [68]:
select.ct %in% levels(park.metadata.df$cluster)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE

In [69]:
levels(park.metadata.df$cluster) %in% select.ct

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE

In [70]:
# Estimate cell type proportions
Est.prop <- music_prop(bulk.eset = craciun.es, 
                      sc.eset = park.filter.sampled.es, 
                      clusters = 'cluster',
                      samples = 'sampleID',
                      select.ct = select.ct, verbose = T)

Creating Relative Abudance Matrix...

Creating Variance Matrix...

Creating Library Size Matrix...

Used 13967 common genes...

Used 16 cell types in deconvolution...

GSM1591197_Normalrep1 has common genes 13051 ...

GSM1591198_Normalrep2 has common genes 13309 ...

GSM1591199_Normalrep3 has common genes 13162 ...

GSM1591200_FA1dayrep1 has common genes 12959 ...

GSM1591201_FA1dayrep2 has common genes 13214 ...

GSM1591202_FA1dayrep3 has common genes 13386 ...

GSM1591203_FA2dayrep1 has common genes 13264 ...

GSM1591204_FA2dayrep2 has common genes 12239 ...

GSM1591205_FA2dayrep3 has common genes 13045 ...

GSM1591206_FA3dayrep1 has common genes 13226 ...

GSM1591207_FA3dayrep2 has common genes 13188 ...

GSM1591208_FA3dayrep3 has common genes 13289 ...

GSM1591209_FA7dayrep1 has common genes 13195 ...

GSM1591210_FA7dayrep2 has common genes 13323 ...

GSM1591211_FA7dayrep3 has common genes 13358 ...

GSM1591212_FA14dayrep1 has common genes 13392 ...

GSM1591213_FA14dayrep2 has comm